In [1]:
from agents.llms.llm import LlamaCppChatCompletion
from agents.tools import get_current_weather, wikipedia_search
from rich import print

In [2]:
llm = LlamaCppChatCompletion.from_default_llm(n_ctx=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
llm.bind_tools([get_current_weather, wikipedia_search])

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that translates English to French."},
    {"role": "user", "content": "Translate this sentence from English to French. I love programming."}
]

print(llm.chat(messages))

ChatCompletion(
    id='chatcmpl-81f7f2d4-76ec-45e8-b4a0-acd843c0e119',
    object='chat.completion',
    created=1715124163,
    model='/Users/aniket/.cache/huggingface/hub/models--meetkai--functionary-small-v2.2-GGUF/snapshots/9971ff5d1eac
bfac81fe6939bccd59cf7d668078/./functionary-small-v2.2.q4_0.gguf',
    choices=[
        Choice(
            index=0,
            logprobs=None,
            message=Message(
                role='assistant',
                content='Certainly! Here\'s the translation:\n\n"Je t\'aime le programme."',
                tool_calls=None
            ),
            finish_reason='stop'
        )
    ],
    usage=Usage(prompt_tokens=317, completion_tokens=24, total_tokens=338)
)

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": "What is the weather in London?"}
]
output = llm.chat(messages)
tool_results = llm.run_tool(output)
print(tool_results)

[
    {
        'tool_call_id': 'call_Hva9ROosHQjWBT9NNuzqO8GV',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': {
            'FeelsLikeC': '16',
            'FeelsLikeF': '61',
            'cloudcover': '0',
            'humidity': '72',
            'localObsDateTime': '2024-05-07 09:23 PM',
            'observation_time': '08:23 PM',
            'precipInches': '0.0',
            'precipMM': '0.0',
            'pressure': '1026',
            'pressureInches': '30',
            'temp_C': '16',
            'temp_F': '61',
            'uvIndex': '1',
            'visibility': '10',
            'visibilityMiles': '6',
            'weatherCode': '113',
            'weatherDesc': [{'value': 'Clear'}],
            'weatherIconUrl': [{'value': ''}],
            'winddir16Point': 'NE',
            'winddirDegree': '40',
            'windspeedKmph': '9',
            'windspeedMiles': '6'
        }
    }
]

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": "Who won the 2024 mayor election in London?"}
]
output = llm.chat(messages)
tool_results = llm.run_tool(output)
print(tool_results)

[
    {
        'tool_call_id': 'call_oO9dod02J2dhdHGED3XCiamV',
        'role': 'tool',
        'name': 'wikipedia_search',
        'content': 'Page: 2024 London mayoral election\nSummary: The 2024 London mayoral election was held on 2 May
2024 to elect the next mayor of London. It took place simultaneously with elections to the London Assembly, some 
local council by-elections in London and regular local elections elsewhere in England and Wales. Following the 
Elections Act 2022, voting in this election took place under the first-past-the-post system for the first time, 
replacing the supplementary vote system. The results of the election were announced on 4 May 2024.\nSadiq Khan of 
the Labour Party, who has served as the mayor of London since 2016, was re-elected as mayor and became the first 
person to be elected to the role for three terms. The Conservative Party stood former London Assembly Conservative 
leader Susan Hall as their candidate, the Green Party nominated Hackney borough councillor Zoë Garbett, the Liberal
Democrats nominated Rob Blackie, and Reform UK nominated Howard Cox. There were eight other candidates.\n\n'
    }
]

In [7]:
updated_messages = messages + tool_results

In [8]:
print(updated_messages)

[
    {
        'role': 'system',
        'content': 'You are a helpful assistant that has access to tools and use that to help humans.'
    },
    {'role': 'user', 'content': 'Who won the 2024 mayor election in London?'},
    {
        'tool_call_id': 'call_oO9dod02J2dhdHGED3XCiamV',
        'role': 'tool',
        'name': 'wikipedia_search',
        'content': 'Page: 2024 London mayoral election\nSummary: The 2024 London mayoral election was held on 2 May
2024 to elect the next mayor of London. It took place simultaneously with elections to the London Assembly, some 
local council by-elections in London and regular local elections elsewhere in England and Wales. Following the 
Elections Act 2022, voting in this election took place under the first-past-the-post system for the first time, 
replacing the supplementary vote system. The results of the election were announced on 4 May 2024.\nSadiq Khan of 
the Labour Party, who has served as the mayor of London since 2016, was re-elected as mayor and became the first 
person to be elected to the role for three terms. The Conservative Party stood former London Assembly Conservative 
leader Susan Hall as their candidate, the Green Party nominated Hackney borough councillor Zoë Garbett, the Liberal
Democrats nominated Rob Blackie, and Reform UK nominated Howard Cox. There were eight other candidates.\n\n'
    }
]

In [9]:
messages = updated_messages + [{"role": "user", "content": "please answer me, based on the tool results."}]
output = llm.chat(messages)
print(output)

ChatCompletion(
    id='chatcmpl-7e96278f-9b13-4607-bdaf-4f1a7b737042',
    object='chat.completion',
    created=1715124169,
    model='/Users/aniket/.cache/huggingface/hub/models--meetkai--functionary-small-v2.2-GGUF/snapshots/9971ff5d1eac
bfac81fe6939bccd59cf7d668078/./functionary-small-v2.2.q4_0.gguf',
    choices=[
        Choice(
            index=0,
            logprobs=None,
            message=Message(
                role='assistant',
                content='The 2024 London mayoral election was held on May 2, 2024. Sadiq Khan of the Labour Party 
won the election and became the first person to be elected to the role for three terms.',
                tool_calls=None
            ),
            finish_reason='stop'
        )
    ],
    usage=Usage(prompt_tokens=589, completion_tokens=53, total_tokens=639)
)